In [10]:
!git clone https://github.com/AlexeyAB/darknet.git


fatal: destination path 'darknet' already exists and is not an empty directory.


In [12]:
%cd darknet
!make


[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh


In [13]:
!wget https://pjreddie.com/media/files/yolov3.weights


--2023-11-30 08:51:54--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  47.0MB/s    in 6.0s    

2023-11-30 08:52:00 (39.6 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [14]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [15]:
import pandas as pd
df =  pd.read_csv('video_frame_mapping.csv')

In [16]:
df.columns

Index(['video_path', 'frame_path', 'category'], dtype='object')

In [21]:
import cv2
import numpy as np
import pandas as pd

# Assuming df is the DataFrame with columns 'video_path', 'frame_path', and 'category'
for index, row in df.iterrows():
    video_path = row['video_path']
    frame_path = row['frame_path']
    category = row['category']

    # Open video
    cap = cv2.VideoCapture(video_path)

    # Get video information
    width = int(cap.get(3))
    height = int(cap.get(4))

    # Create video writer for object detection results
    out = cv2.VideoWriter(f'output_{index}.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (width, height))

    # Load YOLO with pre-trained weights from OpenCV samples
    net = cv2.dnn.readNetFromDarknet(cv2.samples.findFile('yolov3.cfg'), cv2.samples.findFile('yolov3.weights'))
    layer_names = net.getUnconnectedOutLayersNames()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect objects in the frame
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(layer_names)

        # Process detected objects
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Extract object details and add to DataFrame
                    object_name = str(class_id)
                    # You can add more information like confidence, bounding box coordinates, etc.

                    # Create a row for the DataFrame
                    object_data = {
                        'video_path': video_path,
                        'frame_path': frame_path,
                        'category': category,
                        'object_name': object_name,
                        # Add more columns as needed
                    }

                    # Append the object data to the DataFrame
                    object_df = pd.DataFrame([object_data])
                    object_df.to_csv('object_detection_results.csv', mode='a', header=not os.path.exists('object_detection_results.csv'), index=False)

        # Write frame with object detection to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()

cv2.destroyAllWindows()


error: ignored